In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
def to_sequence(data, seq_size=10):
    x, y = [], []

    for i in range(len(data)-seq_size):
        window = data[i:(i+seq_size)].values
        after_window = data[i+seq_size]
        x.append(window)
        y.append(after_window)

    return np.array(x), np.array(y)

In [3]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import *

vertical_MAE =[]
vertical_MSE=[]
vertical_RMSE =[]
vertical_MRE=[]
vertical_MMRE =[]
vertical_PRED=[]

for j in range(1,40):
  # Read csv file into a pandas dataframe
  df = pd.read_csv(f'/content/Patient_{j}.csv')
  print(j)

  for i in range(len(df)):
    if np.isnan(df.at[i,'SensorGLU']):
      X, y = to_sequence(df[:i-1].SensorGLU)
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
      # Create a SVR model
      model = SVR(kernel='linear',gamma='auto', C=1.0, epsilon = 0.1)
      # Train the model
      model.fit(X_train, y_train)
      # Make Predictions
      df.at[i,'SensorGLU']= model.predict((df.loc[i-10:i-1, "SensorGLU"].values).reshape(1,10))


  # Model Assessing
  pred = model.predict(X_test)
  vertical_MAE =np.append(vertical_MAE , mean_absolute_error(y_test,pred))
  vertical_MSE =np.append(vertical_MSE , mean_squared_error(y_test,pred))
  vertical_RMSE =np.append(vertical_RMSE , np.sqrt(mean_squared_error(y_test,pred)))
  vertical_MRE = np.append(vertical_MRE , np.abs((y_test-pred)/y_test)*100)
  vertical_MMRE = np.append(vertical_MMRE , np.mean(np.abs((y_test-pred)/y_test)*100))
  Pred = lambda p: len(vertical_MRE[vertical_MRE <= p])/len(vertical_MRE)*100
  PRED = Pred(25)
  vertical_PRED =np.append(vertical_PRED , PRED)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [4]:
df

,ReadingDateTime,SensorGLU
0,2000-02-10 14:21:00,67.000000
1,2000-02-10 14:26:00,76.000000
2,2000-02-10 14:31:00,86.000000
3,2000-02-10 14:36:00,98.000000
4,2000-02-10 14:41:00,102.000000
...,...,...
343,2000-02-11 18:56:00,58.000000
344,2000-02-11 19:01:00,54.000000
345,2000-02-11 19:06:00,53.876210
346,2000-02-11 19:11:00,59.131033


In [5]:
print(vertical_MAE)

[3.2948332  4.21482917 4.7303147  7.71046549 6.63414016 8.0210638
 7.6268906  4.28111168 6.237333   6.12960155 5.50353149 6.23701331
 9.87567165 3.35608405 2.7633045  7.82665462 4.60618421 9.91568699
 8.69026412 5.85537839 5.34368457 7.76722796 4.299564   4.79390226
 5.57035382 6.67323551 7.14296813 8.27586247 9.46780281 4.30146092
 4.81668352 4.68361888 4.50647745 4.4126688  6.65892875 4.47846742
 7.26935126 7.18754916 7.91908742]


In [6]:
np.median(vertical_MAE)

6.12960154748485

In [7]:
len(vertical_MAE)

39

In [8]:
vertical_RMSE

array([ 5.26906697,  6.83047569,  6.83943989, 12.21613689, 10.14259251,
       12.96600849, 12.83457307,  8.52524694, 11.96681841,  9.13908114,
        9.05561381,  9.40936512, 21.82748453,  5.32875087,  3.85733988,
       11.29296676,  7.2555464 , 16.02286121, 16.85054326,  8.96400473,
        8.65288288, 12.34859813,  9.18740799,  8.54220252,  8.312518  ,
        9.61841757, 13.32053738, 14.52834325, 14.49465721,  6.54142811,
        7.84906375,  6.38192241,  7.86406699,  6.77974736, 10.11431905,
        7.87301707, 10.87173825, 10.97261189, 13.27652268])

In [9]:
np.median(vertical_RMSE)

9.187407987160693

In [12]:
model.score(X_test,y_test)

0.9601092271621766

In [13]:
import csv


test = pd.DataFrame({"MAE": vertical_MAE, "RMSE": vertical_RMSE, "MMRE": vertical_MMRE, "PRED": vertical_PRED})
test.to_csv(r'/content/Patient_result_SVR_30_part1.csv', index=False)